In [1]:
from UniModalReps import *
from sklearn.manifold import TSNE
import torchvision.models as  models
import torch

In [2]:
resnet_50 = models.resnet50(pretrained=True)
res50_C4 = Resnet_C4(resnet_50)
    
# Setting variables for data loading
data_root = '/home/jingwen/data/refcoco'  # contains refclef, refcoco, refcoco+, refcocog and images
dataset = 'refcoco+'
splitBy = 'unc'
refer = REFER(data_root, dataset, splitBy)

loading dataset refcoco+ into memory...
creating index...
index created.
DONE (t=8.04s)


In [4]:
load_statistics(refer)
splits = make_split(dataset)
    
ref_ids_dict = get_refs_id_dict(refer,splits)
refs_dict = get_refslist(refer,ref_ids_dict)
    
refs_small = refs_dict['train'][:10]
ref_id_small = ref_ids_dict['train'][:10]
    
for i in refs_small:
    show_image_annoted(refer,i)

141564 expressions for 49856 refs in 19992 images.
1. navy blue shirt
2. woman back in blue
3. blue shirt
1. gray shirt wearing glasses
2. lady with glasses
3. woman with gray shirt standing next to man
1. standing with racket
2. woman standing
3. holding raquet
1. woman sitting with leg extended
2. ditting darker hair
1. child with case on head
2. holding a green object
3. hand of person with red shirt
1. guy in brown shirt
2. man in brown
3. man wearing long sleeves
1. urinal closest to the white pipe
1. pisser far rat
1. vase holding tallest flower
2. the vase with the tallest flower
3. no 3
1. vase of flower drooping
2. vase with wilted flower
3. vase of the flower tilting over
